### This notebook is to generate df of Positions, that is resulted in our strategy

In [1]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data

### Query Functions

In [2]:
def get_cprice_tick_at_mint(token0, token1, feeTier, mint_time, decimal0, decimal1):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = tick_to_price(ctick,decimal0, decimal1)
    
    return cprice, ctick

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

### Positions Genrations

In [3]:
def get_positions_fixed_strategy(mint_time, burn_time, upper_, lower_, 
                                 decimal0_, decimal1_, feetier_, 
                                 mint_amount0, mint_amount1, mint_tick):
    
    df_positions = pd.DataFrame()
    df_positions['periodStartUnix'] = list(range(mint_time, burn_time+1, 3600))
    df_positions['upper'] = upper_
    df_positions['lower'] = lower_
    df_positions['upper_tick'] = price_to_tick(upper_, decimal0_, decimal1_)
    df_positions['lower_tick'] = price_to_tick(lower_, decimal0_, decimal1_)
    
    df_positions['mint_amount0'] = mint_amount0
    df_positions['mint_amount1'] = mint_amount1
    df_positions['decimal0'] = decimal0_
    df_positions['decimal1'] = decimal1_
    df_positions['feeTier'] = int(feetier_)    
    df_positions['mint_tick'] = int(mint_tick)
    
    return df_positions

### Calculations Functions

In [4]:
def price_to_tick(price_, decimal0_, decimal1_):
    # convert price (token1 by token0) to tick
    tick_ = math.floor( math.log(price_*10**(decimal1_-decimal0_), 1.0001) )
    return tick_

def tick_to_price(tick_,decimal0_, decimal1_):
    price_ = (1.0001**tick_/10**(decimal1_-decimal0_))
    return price_

In [5]:
def check_inRange(ctick, lower_tick, upper_tick):

    if int(ctick) >= int(upper_tick): # change from > to >=
        inRange = 0
    elif int(ctick) < int(lower_tick):
        inRange = 0
    else: 
        inRange = 1
    
    return inRange

In [ ]:
# # Calculate Fees
# # TODO: Clean this up (make it DF operations, make it just 1 function)


# def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amount0_p, amount1_p, amt0_, amt1_, decimal0_, decimal1_, feeTier, mint_tick):
#     # (lower_tick_, upper_tick_) from df_positions
#     # (ctick, L_Pool, swap_volume, amount0_p, amount1_p) from df_merged
#     # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
#     # (decimal0, decimal1, feetier, mint_tick) - fixeds
    
#     mint_liquidity = liq_amounts.get_liquidity(mint_tick, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
#     L_you_ = mint_liquidity # always constant, according to JNP
    
#     inRange_ = check_inRange(ctick_, lower_tick_,upper_tick_)
    
#     if inRange_:
#         pool_share_ = (L_you_/L_Pool_)
#         pool_fee_rate_ = float(feeTier) / 10000
#         fees_0 = pool_share_ * amount0_p * pool_fee_rate_ / 100
#         fees_1 = pool_share_ * amount1_p * pool_fee_rate_ / 100
#     else:
#         fees_0 = 0
#         fees_1 = 0
#         pool_share_ = 0
    
#     return fees_0, fees_1, pool_share_, L_you_

# def calculate_pnl_from_positions(df_merged, df_positions):
#     # TODO: re-calculate amount0 and amount1 to account for every swaps, not just deltas of that hour
#     # use amount0_new and amount1_new instead of amountUSD, and get total accured fees in tokens
#     # Swap_amountIn > so the fees is get from the (+) positive tokens
#     # Therefore, fees you get is in 
#     # Convert those tokens into value (convert using TODAY's value or mints's value?)
    
#     merge_feat = ['periodStartUnix', 'tick','liquidity', 'amountUSD', 'amount0_p', 'amount1_p']

#     df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
#     df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

#     df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
#     for index, row in df_positions_fees.iterrows():
#         row = row.copy()
#         fees_0, fees_1, pool_share, L_you = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],
#                         float(row['liquidity']),row['amountUSD'],
#                         row['amount0_p'], row['amount1_p'],
#                         MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER, MINT_TICK)
       
#         df_positions_fees.loc[index, 'fees_0'] = fees_0
#         df_positions_fees.loc[index, 'fees_1'] = fees_1
#         df_positions_fees.loc[index, 'pool_share'] = pool_share
#         df_positions_fees.loc[index, 'L_you'] = L_you
        
#     return df_positions_fees

In [28]:
def get_fees_from_positions(df_merged_, df_positions_):
    
    merge_feat_ = ['periodStartUnix', 'tick','liquidity','sqrtPrice', 
                   'feeGrowthGlobal0X128', 'feeGrowthGlobal1X128',
                   'amountUSD', 'amount0_p', 'amount1_p']
    df_ = df_positions_.merge(df_merged_[merge_feat_], how='left',on='periodStartUnix')
    df_['liquidity'] = df_['liquidity'].astype(float) # as float
    df_['sqrtPrice'] = df_['sqrtPrice'].astype(float)
    df_['feeGrowthGlobal0X128'] = df_['feeGrowthGlobal0X128'].astype(float)
    df_['feeGrowthGlobal1X128'] = df_['feeGrowthGlobal1X128'].astype(float)
    
    # CAREFUL: 
    print('Shape Before dropNA: {}'.format(df_.shape))
    df_ = df_.dropna(subset=['tick','liquidity'])
    print('Shape After dropNA: {}'.format(df_.shape))
    
    # assigns features neccessary for fees calculation
    df_['L_you'] = df_.apply(lambda x: liq_amounts.get_liquidity(x['mint_tick'], x['lower_tick'],
                                            x['upper_tick'], x['mint_amount0'], x['mint_amount1'],
                                            x['decimal0'], x['decimal1']), axis=1)
#     df_['pool_share'] = df_['L_you']/df_['liquidity']
    df_['pool_share'] = df_['L_you'] / ( df_['liquidity'] + df_['L_you'] )
    df_['inRange'] = df_.apply(lambda x: check_inRange(x['tick'], 
                                            x['lower_tick'], x['upper_tick']),axis=1)
    df_['fees0'] = df_['inRange'] * df_['pool_share'] * df_['amount0_p'] * df_['feeTier'] / (10000*100)
    df_['fees1'] = df_['inRange'] * df_['pool_share'] * df_['amount1_p'] * df_['feeTier'] / (10000*100)

    # fix something with decimals? is divide by 2**128 correct?
    df_['fees0_G'] = df_['inRange'] * ( df_['feeGrowthGlobal0X128'] * df_['pool_share'] ) / (2**128)
    df_['fees1_G'] = df_['inRange'] * ( df_['feeGrowthGlobal1X128'] * df_['pool_share']) / (2**128)
    
    return df_

In [29]:
df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627981200-1620169200.csv')
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
                                            DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
                                           MINT_AMT1, MINT_TICK)

In [30]:
df_positions_fees = get_fees_from_positions(df_merged, df_positions)

fees_token0 = df_positions_fees['fees0'].sum()
fees_token1 = df_positions_fees['fees1'].sum()

fees_token0_G = df_positions_fees['fees0_G'].sum()
fees_token1_G = df_positions_fees['fees1_G'].sum()

print(fees_token0, fees_token1)
print(fees_token0_G, fees_token1_G)

# totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

# print(fees_token0, fees_token1, totalfees_USD)
# print(revert_fees0,revert_fees1)

# error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
# error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
# print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

Shape Before dropNA: (1556, 19)
Shape After dropNA: (1556, 19)
35747.20051661218 16.023636054616944
7.830100949913882e-06 3060.3758777550197


### Main

In [ ]:
# STEP 0:  PROVIDE BY USERS
UPPER = 1/1844.6164 # 1/1844.6164  # token1 per token0 (in terms of price)
LOWER = 1/2858.3641 # 1/2858.3641  # 1/2376.6075 
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
# needs to start with the hour (unixTime % 3600 = 0)
MINT_TIME = 1622304000 # 4PM May 29 (UTC/GMT)
BURN_TIME = 1627866000 # 1AM Aug 02 (UTC/GMT)

# STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# THIS CHANGES over time when swaps happens, and we need this to calculate IL
MINT_AMT0 = 252208.7053
MINT_AMT1 = 99.0604

In [ ]:
# Get required constants
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(DECIMAL0, DECIMAL1)

MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
print(MINT_PRICE, 1/MINT_PRICE)
print(MINT_TICK)

In [ ]:
# STEP 3 (TODO): Generate df_positions
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
                                            DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
                                           MINT_AMT1, MINT_TICK)

In [14]:
# Get DF_Merged (accumulated swaps data)
# df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627873200-1620169200.csv')
df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}
 

 Queried poolHourDatas, total of 2160 datapoints
time_start: 1627981200, time_end: 1620169200
next_time_start_: 1627981200
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1627261261
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1626450194
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1625627459
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1624908123
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1624418849
Queried Swaps, total of 6000 datapoints
 
next_time_sta

In [ ]:
df_merged.info()

In [ ]:
# STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)

In [ ]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [ ]:
TOKEN0_TO_USD = 1
TOKEN1_TO_USD = 2561.94

fees_token0 = df_positions_fees['fees_0'].sum()
fees_token1 = df_positions_fees['fees_1'].sum()

totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

print(fees_token0, fees_token1, totalfees_USD)
revert_fees0 = 37795.612553
revert_fees1 = 16.845536
error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

### Testing margin of Errors

In [7]:
# References: 
# 1. https://revert.finance/#/uniswap-position/27782
# 2. https://revert.finance/#/uniswap-position/2067
# 3. https://revert.finance/#/uniswap-position/19724

UPPERS = [1/1844.6164 , 1.0014, 1/1655.7876]
LOWERS = [1/2858.3641 , 1.0004, 1/3334.2300] 
FEETIERS = ['3000' , '500', '10000']
TOKEN0S = ['USDC' , 'DAI', 'USDC']
TOKEN1S = ['WETH' , 'USDC', 'WETH']
MINT_TIMES = [1622304000, 1621720800, 1621641600] # _____ ,may6th, may22nd
BURN_TIMES = [1627902000, 1627902000, 1627894800] # NOWS

MINT_AMOUNT0S = [252208.7053 , 13068.6711, 771.4900]
MINT_AMOUNT1S = [99.0604 , 12194.3375, 0.2697]

TOKEN0_TO_USDS = [1, 1, 1 ]
TOKEN1_TO_USDS = [2578.84 , 1, 2578.84 ]

REVERT_FEES0 = [37795.612553 , 421.148243, 136.562165]
REVERT_FEES1 = [16.845536 , 420.589287, 0.060010 ]

In [8]:
i = 0

UPPER = UPPERS[i]
LOWER = LOWERS[i] 
FEETIER = FEETIERS[i]
TOKEN0 = TOKEN0S[i]
TOKEN1 = TOKEN1S[i]

MINT_TIME = MINT_TIMES[i] 
BURN_TIME = BURN_TIMES[i] 

MINT_AMT0 = MINT_AMOUNT0S[i]
MINT_AMT1 = MINT_AMOUNT1S[i]

TOKEN0_TO_USD = TOKEN0_TO_USDS[i]
TOKEN1_TO_USD = TOKEN1_TO_USDS[i]

In [9]:
# Get required constants
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(DECIMAL0, DECIMAL1)

MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
print(MINT_PRICE, 1/MINT_PRICE)
print(MINT_TICK)

revert_fees0 = REVERT_FEES0[i]
revert_fees1 = REVERT_FEES1[i]

6.0 18.0
 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}
0.00042136376404820986 2373.2463142834135
198600


In [10]:
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
                                            DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
                                           MINT_AMT1, MINT_TICK)

In [11]:
df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)
# df_merged = df_merged.fillna(0)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}
 

 Queried poolHourDatas, total of 2160 datapoints
time_start: 1627981200, time_end: 1620169200
next_time_start_: 1627981200
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1627261261
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1626450194


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-3b2e48134348>", line 1, in <module>
    df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)
  File "/Users/ren10x/Desktop/Desktop - Ren’s MacBook Pro/schoolbus/code/backtest/load_data.py", line 345, in get_data
    df_swaps_all = get_swaps_loop(pool_id, time_start, time_end) # ,  num_datapoints= 150000
  File "/Users/ren10x/Desktop/Desktop - Ren’s MacBook Pro/schoolbus/code/backtest/load_data.py", line 242, in get_swaps_loop
    df_, next_time_start_ = get_swaps(pool_id, next_time_start_, time_end, num_datapoints=max_num_query)
  File "/Users/ren10x/Desktop/Desktop - Ren’s MacBook Pro/schoolbus/code/backtest/load_data.py", line 204, in get_swaps
    query_result_ = run_query(query_)
  File "/Users/ren10x/Desktop/Desktop - Ren’s

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_positions_fees = get_fees_from_positions(df_merged, df_positions)

In [ ]:
# df_positions_fees = get_fees_from_positions(df_merged, df_positions)

fees_token0 = df_positions_fees['fees0'].sum()
fees_token1 = df_positions_fees['fees1'].sum()

totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

print(fees_token0, fees_token1, totalfees_USD)
print(revert_fees0,revert_fees1)

error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

In [ ]:
# 35835.9997570211 16.065701314551056 77266.87293503794
# error margin token0 = 5.184762631458817 %, token1 = 4.629325451258682 %


# 167.380859180713 0.07378978912937459 357.67291897910934
# error margin token0 = -22.567520206429805 %, token1 = -22.9624881342686 %

In [ ]:
watch = ['periodStartUnix', 'feeGrowthGlobal0X128', 'feeGrowthGlobal1X128']
df_positions_fees[watch].head()